<center><img src="./images/logo.png" style="width: 300px;"></center>

# Fruit Classification Notebook Workflow
This notebook demonstrates a complete workflow for classifying fresh and rotten fruits using deep learning.

#### Problem Statement

A leading fruit supply company has approached **FruitScan Solutions** to develop an automated solution that can distinguish between fresh and rotten fruits in real-time using image input from their supply belt. The supplier, primarily dealing in apples, oranges, and bananas, faces challenges in ensuring consistent fruit quality, often resulting in customer complaints due to the inclusion of rotten fruits in purchased baskets. Manual sorting is slow, inconsistent, and not scalable. The company seeks to increase the speed and accuracy of their quality control process by integrating a computer vision-based system that can automate fruit classification and help streamline their distribution workflow

## Importing Required Libraries
Essential libraries for deep learning (PyTorch, torchvision), data handling, and image processing are imported. The device (CPU or GPU) is set for computation.

In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.v2 as transforms
import torchvision.io as tv_io
import os
import glob
from PIL import Image

import utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()


True

## Downloading and Preparing the Dataset
   The fruit image dataset is downloaded using KaggleHub. The dataset contains labeled images of fresh and rotten fruits, organized into training and validation folders.


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sriramr/fruits-fresh-and-rotten-for-classification")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fruits-fresh-and-rotten-for-classification


## Visualizing the Dataset
   A sample image from the dataset is displayed to provide a visual understanding of the data.


<center><img src="./images/fruits.png" style="width: 600px;"></center>

## Model Selection and Initialization
   A pre-trained VGG16 model is loaded from torchvision. The base model's weights are frozen to leverage transfer learning, and the classifier is customized for the fruit classification task.


In [3]:
from torchvision.models import vgg16
from torchvision.models import VGG16_Weights

weights = VGG16_Weights.DEFAULT
vgg_model = vgg16(weights=weights)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:07<00:00, 77.6MB/s]


In [4]:
# Freeze base model
vgg_model.requires_grad_(False)
next(iter(vgg_model.parameters())).requires_grad

False

In [5]:
vgg_model.classifier[0:3]

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
)

## Model Customization
   The classifier part of the VGG16 model is modified to output predictions for six classes (fresh and rotten for each fruit type). Additional fully connected layers and activation functions are added.


In [6]:
N_CLASSES = 6

my_model = nn.Sequential(
    vgg_model.features,
    vgg_model.avgpool,
    nn.Flatten(),
    vgg_model.classifier[0:3],
    nn.Linear(4096, 500),
    nn.ReLU(),
    nn.Linear(500, N_CLASSES)
)
my_model

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## Setting Up Loss Function and Optimizer
   The cross-entropy loss function is used for multi-class classification. The Adam optimizer is initialized to update the model parameters during training.


In [7]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(my_model.parameters())
my_model = torch.compile(my_model.to(device))

## Data Transformations and Augmentation
   Image transformations such as resizing, rotation, and horizontal flipping are applied to augment the training data and improve model generalization.


In [8]:
pre_trans = weights.transforms()

In [9]:
IMG_WIDTH, IMG_HEIGHT = (224, 224)

random_trans = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop((IMG_WIDTH, IMG_HEIGHT), scale=(.8, 1), ratio=(1, 1)),
    transforms.RandomHorizontalFlip(),
])

## Custom Dataset Class
   A custom PyTorch Dataset class is defined to load images and labels from the dataset folders, apply transformations, and prepare data for training and validation.


In [10]:
DATA_LABELS = ["freshapples", "freshbanana", "freshoranges", "rottenapples", "rottenbanana", "rottenoranges"]

class MyDataset(Dataset):
    def __init__(self, data_dir):
        self.imgs = []
        self.labels = []

        for l_idx, label in enumerate(DATA_LABELS):
            data_paths = glob.glob(data_dir + label + '/*.png', recursive=True)
            for path in data_paths:
                img = tv_io.read_image(path, tv_io.ImageReadMode.RGB)
                self.imgs.append(pre_trans(img).to(device))
                self.labels.append(torch.tensor(l_idx).to(device))


    def __getitem__(self, idx):
        img = self.imgs[idx]
        label = self.labels[idx]
        return img, label

    def __len__(self):
        return len(self.imgs)

## DataLoader Setup
   PyTorch DataLoaders are created for both training and validation datasets to efficiently batch and shuffle data during model training and evaluation.


In [ ]:
n = 32

In [24]:
train_path = '/kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/train/'
train_data = MyDataset(train_path)
train_loader = DataLoader(train_data, batch_size=n, shuffle=True)
train_N = len(train_loader.dataset)

In [27]:
valid_path = '/kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/test/'
valid_data = MyDataset(valid_path)
valid_loader = DataLoader(valid_data, batch_size=n, shuffle=False)
valid_N = len(valid_loader.dataset)

In [28]:
train_N, valid_N

(10901, 2698)

## Model Training
   The model is trained for several epochs using the training data. After each epoch, the model's performance is validated on the validation set to monitor progress and prevent overfitting.


In [29]:
epochs = 15

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    utils.train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
    utils.validate(my_model, valid_loader, valid_N, loss_function)

Epoch: 0
Train - Loss: 21.9565 Accuracy: 0.9764
Valid - Loss: 5.1158 Accuracy: 0.9822
Epoch: 1
Train - Loss: 18.1268 Accuracy: 0.9824
Valid - Loss: 4.0402 Accuracy: 0.9811
Epoch: 2
Train - Loss: 13.0671 Accuracy: 0.9861
Valid - Loss: 2.1111 Accuracy: 0.9900
Epoch: 3
Train - Loss: 13.0427 Accuracy: 0.9880
Valid - Loss: 2.6491 Accuracy: 0.9911
Epoch: 4
Train - Loss: 11.4938 Accuracy: 0.9871
Valid - Loss: 1.7292 Accuracy: 0.9937
Epoch: 5
Train - Loss: 10.0478 Accuracy: 0.9906
Valid - Loss: 1.9499 Accuracy: 0.9922
Epoch: 6
Train - Loss: 13.7807 Accuracy: 0.9859
Valid - Loss: 3.1426 Accuracy: 0.9870
Epoch: 7
Train - Loss: 8.0254 Accuracy: 0.9918
Valid - Loss: 2.4182 Accuracy: 0.9896
Epoch: 8
Train - Loss: 9.2837 Accuracy: 0.9907
Valid - Loss: 2.3325 Accuracy: 0.9904
Epoch: 9
Train - Loss: 7.5666 Accuracy: 0.9932
Valid - Loss: 2.2021 Accuracy: 0.9900
Epoch: 10
Train - Loss: 7.4638 Accuracy: 0.9923
Valid - Loss: 1.5260 Accuracy: 0.9937
Epoch: 11
Train - Loss: 7.7227 Accuracy: 0.9929
Valid - L

## Fine-Tuning the Model
   After initial training, the base VGG16 model is unfrozen to allow fine-tuning of all layers. The optimizer's learning rate is reduced, and the model is trained for additional epochs to further improve accuracy.


In [30]:
# Unfreeze the base model
vgg_model.requires_grad_(True)
optimizer = Adam(my_model.parameters(), lr=.0001)

In [33]:
epochs = 5

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    utils.train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
    utils.validate(my_model, valid_loader, valid_N, loss_function)

Epoch: 0
Train - Loss: 2.6605 Accuracy: 0.9972
Valid - Loss: 1.7653 Accuracy: 0.9944
Epoch: 1
Train - Loss: 3.1402 Accuracy: 0.9975
Valid - Loss: 1.5938 Accuracy: 0.9948
Epoch: 2
Train - Loss: 2.3308 Accuracy: 0.9972
Valid - Loss: 1.2963 Accuracy: 0.9948
Epoch: 3
Train - Loss: 2.5016 Accuracy: 0.9974
Valid - Loss: 1.4688 Accuracy: 0.9956
Epoch: 4
Train - Loss: 2.2837 Accuracy: 0.9976
Valid - Loss: 1.3724 Accuracy: 0.9956


## Model Evaluation
   The final trained model is evaluated on the validation set to assess its classification performance. Metrics such as accuracy and loss are reported.

In [34]:
utils.validate(my_model, valid_loader, valid_N, loss_function)

Valid - Loss: 1.3724 Accuracy: 0.9956


## Saving the Model
FInally the FIne-tuned model is saved in the `model` directory.

In [ ]:
# # Create the directory if it doesn't exist
# !mkdir -p model

# # Save the model
# torch.save(my_model.state_dict(), 'model/fruit_classification_model.pth')
